In [ ]:
import pandas as pd
import geopandas as gpd
import requests as re
from io import BytesIO
from os import path, makedirs

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## Orçamento previsto/liquidado no Programa 3002

Vamos começar coletando os dados orçamentários do site de execução orçamentária da Secretaria da Fazenda.

In [ ]:
url_orcamento = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosexecucao_1224.csv'
df_orcamento = pd.read_csv(url_orcamento,
                           sep=';',
                           decimal=',',
                           encoding='latin1',
                           dtype=str)
df_orcamento

In [ ]:
for col in [col for col in df_orcamento.columns if 'Vl' in col]:
    df_orcamento.loc[:, col] = df_orcamento[col].str.replace(',', '.').astype(float)
df_orcamento.loc[:, 'DataExtracao'] = pd.to_datetime(df_orcamento['DataExtracao'], format='%d/%m/%Y')
df_orcamento

In [ ]:
df_orcamento = df_orcamento.loc[df_orcamento['Cd_Funcao']=='16']
df_orcamento

## Orçamento regionalizado no Programa 3002

O orçamento é regionalizado apenas na liquidação, então não é possível obter o orçamento previsto por subprefeitura, mas é possível obter o orçamento liquidado.

In [ ]:
url_orcamento_r = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosDA_1224.csv'
df_orcamento_r = pd.read_csv(url_orcamento_r,
                           sep=';',
                           decimal=',',
                           encoding='latin1',
                           dtype=str)
df_orcamento_r

In [ ]:
df_orcamento_r.loc[:, 'VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace(',', '.').astype(float)
df_orcamento_r.loc[:, 'DATA_EXTRAÇÃO'] = pd.to_datetime(df_orcamento_r['DATA_EXTRAÇÃO'], format='%d/%m/%Y')
df_orcamento_r

In [ ]:
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['CÓDIGO_FUNÇÃO']=='16']
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['ANO_LIQUIDAÇÃO']=='2024']
df_orcamento_r

Vamos conferir qual o percentual do orçamento está presente na tabela de detalhamento e regionalizado a nível de subprefeitura.

In [ ]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()

95,3% de detalhamento é um percentual excelente. Vamos checar a regionalização.

In [ ]:
(
    df_orcamento_r
    .assign(regionalizado=df_orcamento_r['SUBPREFEITURA'].str.contains('Supra')==False)
    .groupby('regionalizado')
    ['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()
)

O percentual de regionalização é de 64,4%. Menor, mas ainda parece ser satisfatório. Vamos manter as informações de regionalização também.

## Cumprimento da Meta 12: Prover 49.000 moradias de interesse social

Esse indicador está disponível de forma regionalizada na página do PdM da Secretaria de Governo Municipal. 

In [ ]:
url_meta_12 = 'https://prefeitura.sp.gov.br/documents/d/planejamento/relatorio-de-final-do-programa-de-metas-2021-2024_marco2025-xlsx'

resp_meta_12 = re.get(url_meta_12)
resp_meta_12.raise_for_status()
xlsx_meta_12 = BytesIO(resp_meta_12.content)
excel_meta_12 = pd.ExcelFile(xlsx_meta_12)

excel_meta_12.sheet_names

In [ ]:
sheet_name = 'regionalização'
df_meta_12 = pd.read_excel(xlsx_meta_12,
                   sheet_name=sheet_name)
df_meta_12

In [ ]:
df_meta_12 = df_meta_12[df_meta_12['Número da Meta']==12]
df_meta_12

## Produção de habitação de interesse social

O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

In [ ]:
url_his = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=285'
df_his = pd.read_csv(url_his,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_his

In [ ]:
df_his = df_his[df_his['nivel_regional']=='Subprefeitura']
df_his

In [ ]:
df_his = df_his.loc[:, [
    'região',
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_his

In [ ]:
df_his = df_his.melt(
        ['região', 'indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_unidades'
    )

df_his

## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [ ]:
url_tpu = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=662'
df_tpu = pd.read_csv(url_tpu,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_tpu

In [ ]:
df_tpu = df_tpu.loc[:, [
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_tpu

In [ ]:
df_tpu = df_tpu.melt(
        ['indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_termos'
    )

df_tpu

## Número de domicílios em favelas

Para utilizarmos os dados de favelas e comunidades urbanas do último censo, primeiro precisamos extrair os dados geográficos do site do IBGE. Como o arquivo contém todas as favelas do Brasil, vamos filtrar apenas as favelas do município de São Paulo.

In [ ]:
url = "https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Favelas_e_comunidades_urbanas_Resultados_do_universo/arquivos_vetoriais/poligonos_FCUs_shp.zip"
df_favelas = gpd.read_file(url)
df_favelas = df_favelas[df_favelas['cd_mun']=='3550308']
df_favelas

Em seguida, vamos conferir o CRS utilizado no arquivo e, caso necessário, converter para o EPSG 31983.

In [ ]:
df_favelas.crs

In [ ]:
df_favelas = df_favelas.to_crs(epsg=31983)
df_favelas

Agora, vemos também que os atributos com a quantidade de domicílios não estão presentes no arquivo. Portanto, precisamos baixar o arquivo de atributos, que está disponível no mesmo site do IBGE.

In [ ]:
url = 'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela9887_35_FAV_COM.xlsx&terr=N&rank=-&query=t/9887/n170/in%20n3%2035/v/allxp/p/all/c3/all/l/v,p%2Bc3,t'
df_favelas_domicilios = pd.read_excel(url,
                                      skiprows=4,
                                      engine='openpyxl',
                                      dtype=str)
df_favelas_domicilios

O arquivo contém todas as favelas e comunidades urbanas do Estado de São Paulo, então vamos filtrar apenas as favelas do município de São Paulo.

In [ ]:
df_favelas_domicilios = df_favelas_domicilios[df_favelas_domicilios.iloc[:,0].str.contains('São Paulo (SP)', regex=False)]
df_favelas_domicilios

In [ ]:
df_favelas_domicilios.shape[0] == df_favelas.shape[0]

Como as duas tabelas possuem a mesma quantidade de linhas, podemos assumir que o filtro foi realizado corretamente.

Vamos também garantir que as colunas de atributos estão todas com o formato de inteiro. Caso existam colunas com formato string, pode significar que existem valores especiais, conforme explicado na planilha "Notas" disponível no arquivo.

| Símbolo | Significado |
|---------|-------------|
| - | Zero absoluto, não resultante de um cálculo ou arredondamento. Ex: Em determinado município não existem pessoas de 14 anos de idade sem instrução. |
| 0 | Zero resultante de um cálculo ou arredondamento. Ex: A inflação do feijão em determinada Região Metropolitana foi 0. Determinado município produziu 400 kg de sementes de girassol e os dados da tabela são expressos em toneladas. |
| X | Valor inibido para não identificar o informante. Ex: Determinado município só possui uma empresa produtora de cimento, logo o valor de sua produção deve ser inibido. |
| .. | Valor não se aplica. Ex: Não se pode obter o total da produção agrícola em determinado município quando os produtos agrícolas são contabilizados com unidades de medida distintas. |
| ... | Valor não disponível. Ex: A produção de feijão em determinado município não foi pesquisada ou determinado município não existia no ano da pesquisa. |
| A a Z (exceto X) | Significa uma faixa de valores. Varia em função da tabela (se for o caso). Ex: O nível de precisão da produção estimada de combustíveis está na faixa A (95 a 100%) |

Primeiro, vamos substituir os valores `-` por `0`, para que a conversão para inteiro funcione corretamente. Caso existam outros valores especiais, como `X`, `..`, ou `...`, devemos avaliar a possibilidade de excluir essas colunas.

In [ ]:
df_favelas_domicilios.dtypes

In [ ]:
for col in df_favelas_domicilios.columns:
    if (df_favelas_domicilios[col].dtype == 'object' and col != df_favelas_domicilios.columns[0]):
        df_favelas_domicilios.loc[:, col] = df_favelas_domicilios.loc[:, col].str.replace('-', '0')
        if df_favelas_domicilios[col].str.contains(r'[^\d]', regex=True).any():
            print(f'Coluna {col} contém valores não numéricos além de "-"')

In [ ]:
dtypes = {col: 'int64'
          for col in df_favelas_domicilios.columns
          if col != df_favelas_domicilios.columns[0]}

dtypes

In [ ]:
df_favelas_domicilios = df_favelas_domicilios.astype(dtypes)
df_favelas_domicilios.dtypes

Agora, podemos juntar as duas tabelas utilizando o campo de nome da favela/comunidade urbana.

Primeiro, vamos extrair apenas o nome da favela/comunidade urbana do campo de nome, que contém o nome do município também.

In [ ]:
df_favelas_domicilios.iloc[:, 0] = df_favelas_domicilios.iloc[:, 0].str.replace(' - São Paulo (SP)', '', regex=False)
df_favelas_domicilios

In [ ]:
df_favelas_domicilios.iloc[:, 0].describe() 

Vemos, também, que todos os nomes são únicos, então podemos prosseguir com o merge sem problemas. Vamos apenas substituir o caractere `´` por `'`, porque seus usos não estão completamente padronizados entre as duas tabelas.

In [ ]:
df_favelas.loc[:, 'nm_fcu'] = df_favelas['nm_fcu'].str.replace('´', "'", regex=False)
df_favelas_domicilios.iloc[:, 0] = df_favelas_domicilios.iloc[:, 0].str.replace('´', "'", regex=False)

In [ ]:
df_favelas = (
    df_favelas
    .merge(
        df_favelas_domicilios,
        left_on='nm_fcu',
        right_on=df_favelas_domicilios.columns[0],
        how='left',
        validate='1:1'
    )
    .drop(columns=[df_favelas_domicilios.columns[0]])
)

df_favelas

In [ ]:
df_favelas.isna().sum()

Todas os registros foram combinados corretamente, então podemos prosseguir para a exportação dos dados.

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [ ]:
output_dir = path.join('data', 'input', 'urbanismo')

if not path.exists(output_dir):
    makedirs(output_dir)

for c, df in [('orcamento_habitacao_original', df_orcamento),
               ('orcamento_regionalizado_habitacao_original', df_orcamento_r),
               ('pdm_meta_12_original', df_meta_12),
               ('his_entregue_original', df_his),
               ('tpu_emitido_original', df_tpu)
               ]:
    filename=path.join(output_dir, c)
    df.to_csv(f'{filename}.csv',
              sep=';',
              decimal=',',
              encoding='utf8',
              index=False
              )
    
df_favelas.to_file(path.join(output_dir, 'favelas_original.gpkg'), driver='GPKG')